## 1. Obtenemos la siguientes medidas:

### Obtenemos las siguientes métricas:
- P(e|d)
- P(d|e)
- ICC
- TScore
- IDF
- IDF normalizado

In [1]:
import os
import pickle
import math
import statistics 

In [2]:
def deserialize_object(path):
    pickle_in = open(path,"rb")
    obj = pickle.load(pickle_in)
    pickle_in.close()
    print("Cargado el objeto", path.split("/")[- 1])
    return obj

## 1.1 Obtenemos el nº de documentos del corpus

In [3]:
list_docs_total = []
for specialty, dic in dic_spe.items():
    list_doc_total_specialty = dic['docs']
    
    list_docs_total.extend(list_doc_total_specialty)

print("Nº de documentos totales:", len(set(list_docs_total)))


NameError: name 'dic_spe' is not defined

## 1.2 Obtenemos las métricas de cada término

In [3]:
# Nº de documentos del corpus
num_total_docs_corpus = 195234

#Nº de palabras en el corpus: 6469046
#Nº de palabras diferentes en el corpus: 115121
N = 6469046

In [4]:
#Nº de documentos en la especialidad que NO tienen el n-gram
def obtain_ndpe(list_doc_term, list_doc_total_spe):
    return len(set(set(list_doc_total_spe)-set(list_doc_term)))

In [5]:
# Et: nº de especialidades que contienen el ngram
def obtain_et(term, dic_spe):
    et = 0

    for specialty, dic in dic_spe.items():
        
        # obtenemos el diccionario de términos
        dic_term = dic['terms']
        
         # si el término se encuentra en el diccionario sumamos 1
        if term in dic_term:
            et += 1
            
    return et      

In [6]:
#Número de documentos en las OTRAS especialidades que tienen el n-gram
def obtain_ndep(term, dic_spe, specialty_actual):
    
    ndpe = 0

    for specialty, dic in dic_spe.items():
        
        # si la especialidad no es la misma
        if not specialty == specialty_actual:
            
            # obtenemos el diccionario de términos
            dic_term = dic['terms']
            
            # si el término se encuentra en el diccionario
            if term in dic_term:
                
                # obtenemos los documentos en los que aparece
                doc_term = list(set(dic_term[term]))
                
                # contamos el nº de documentos y sumamos
                ndpe += len(doc_term)
                
    return ndpe

In [7]:
# obtenemos las veces que se repite el termino en el corpus
def obtain_nocc_term(term, dic_spe):
    
    nocc_term = 0

    for specialty, dic in dic_spe.items():
        # obtenemos el diccionario de términos
        dic_term = dic['terms']

        # si el término se encuentra en el diccionario
        if term in dic_term:

            # obtenemos el nº los documentos en los que aparece
            nocc_term += len(dic_term[term])
                
    return nocc_term

In [8]:
# t-score
def obtain_tscore(term, dic_spe):
    
    # unigramas
    if not type(term) is tuple:
        nocc_term = obtain_nocc_term(term, dic_spe)
        
        ts = nocc_term - nocc_term / math.sqrt(nocc_term / N)
        
        return ts, nocc_term, 0, 0, 0
       
    else:
        # brigramas
        if len(term) == 2:
            nocc_bigram = obtain_nocc_term(term, dic_spe)
            nocc_w1 = obtain_nocc_term(term[0], dic_spe)
            nocc_w2 = obtain_nocc_term(term[1], dic_spe)
        
            ts = nocc_bigram - (nocc_w1 * nocc_w2) / math.sqrt(nocc_bigram / N)
            
            return ts, nocc_bigram, nocc_w1, nocc_w2, 0
            
        # trigramas
        elif len(term) == 3:
            
            nocc_trigram = obtain_nocc_term(term, dic_spe)
            nocc_w1 = obtain_nocc_term(term[0], dic_spe)
            nocc_w2 = obtain_nocc_term(term[1], dic_spe)
            nocc_w3 = obtain_nocc_term(term[2], dic_spe)
            
            ts =  ( nocc_trigram - (nocc_w1 * nocc_w2 * nocc_w3) ) / math.sqrt(nocc_trigram / N)
        
            return ts, nocc_trigram, nocc_w1, nocc_w2, nocc_w3

In [9]:
def obtain_n_of_ngram(term):
    if not type(term) is tuple:
        return 1
    else:
        if len(term) == 2:
            return 2
        elif len(term) == 3:
            return 3

In [10]:
def calculate_weight_terms(path_out, dic_spe):
    
    #list_spe_import = ['H02.403.680.600_rehabilitation', 'H02.403.225_dermatology', 'H02.403.763.750_gynecology', 'H02.403.740_radiology']
    #list_spe_import = ['H02.403.340.500_family_practice']
    for specialty, dic in dic_spe.items():
        
        #if specialty in list_spe_import:
        if not os.path.exists(path_out + specialty + "_terms_weight.csv"):  
            with open(path_out + specialty + "_terms_weight.csv", "w") as fout:

                list_doc_total_specialty = dic['docs']
                dic_terms = dic['terms']

                print("Especialidad:", specialty, 'nº docs total:', len(list_doc_total_specialty))

                # primero obtenemos el idf de todos los términos para calcular la media y desviación
                # para posteriormente calcular el idf-normalizado
                list_idfs = []

                for term, list_doc_term in dic_terms.items():

                    nde = len(set(list_doc_term))
                    ndep = obtain_ndep(term, dic_spe, specialty)

                    num_doc_term = nde + ndep

                    idf = math.log2(num_total_docs_corpus / num_doc_term)

                    list_idfs.append(idf)

                mean_idfs = statistics.mean(list_idfs)
                desv_idfs = statistics.stdev(list_idfs)

                '''
                    Creamos las nuevas métricas
                '''

                fout.write("Término|")

                fout.write("Ngram|")

                fout.write("N(d,e)|")
                fout.write("N(d,e')|")
                fout.write("N(d',e)|")

                fout.write("P(e,d)|")
                fout.write("P(d,e)|")

                fout.write("SpecialtyWeight|")

                fout.write("Idf|")
                fout.write("Idf-norm|")

                fout.write("SpecialtyWeight*Idf-norm|")

                fout.write("\n")

                for term, list_doc_term in dic_terms.items():

                    num_ooc_spe = len(list_doc_term)

                    nde = len(set(list_doc_term))

                    ndep = obtain_ndep(term, dic_spe, specialty)
                    ndpe = obtain_ndpe(list_doc_term, list_doc_total_specialty)
                    #et = obtain_et(term, dic_spe)

                    # p(e|d)
                    ped = nde / (nde + ndep)

                    # p(d|e)
                    pde = nde / (nde + ndpe)

                    # specialty level weight
                    slw = ped + pde

                    # idf
                    num_doc_term = nde + ndep
                    idf = math.log2(num_total_docs_corpus / num_doc_term)

                    # idf-norml
                    idf_norm = (idf - mean_idfs) / desv_idfs

                    # specialty level weight * idf-norml
                    tw = slw * idf_norm

                    # icc
                    #icc = 1 /  et

                    # t-score
                    #tscore, nocc_term, nocc_w1, nocc_w2, nocc_w3 = obtain_tscore(term, dic_spe)
                    #print(nocc_term, nocc_w1, nocc_w2, nocc_w3)

                    # corpus level weight
                    #clw = icc * tscore

                    # total weight
                    #tw = clw * slw

                    # otra prueba (icc x specialty level weight)
                    #other_total = icc * slw

                    # imprimir en fichero
                    fout.write(str(term) + "|")

                    # si es 1 gram 2 gram o 3 gram
                    fout.write(str(obtain_n_of_ngram(term)) + '|')

                    fout.write(str(nde) +'|')
                    fout.write(str(ndep)+ '|')
                    fout.write(str(ndpe) + '|')

                    fout.write(str(ped)+ '|')
                    fout.write(str(pde)+ '|')

                    fout.write(str(slw) + "|")

                    fout.write(str(idf) + "|")
                    fout.write(str(idf_norm) + "|")

                    fout.write(str(tw) + "|")

                    fout.write('\n')


In [11]:
path_dic = 'dic_specialties.pkl'
path_dic_lemma = 'dic_specialties_lemma.pkl'

dic_spe = deserialize_object(path_dic_lemma)

Cargado el objeto dic_specialties_lemma.pkl


In [ ]:
#path_output = './term_weight/1_3grams/'
#path_output = './term_weight/1_5grams/'
path_output = './term_weight/1_3grams_lemma/'

calculate_weight_terms(path_output, dic_spe)    

Especialidad: H02.403.720.750_preventive_medicine nº docs total: 83263
